# DI 725 Project Phase 2 - Ali Yiğit Başaran - 2231355 
#### Dataset Preparation - Modeling - Train, Validation Loop - Test & Log Metrics 

In [ ]:
# Setup and Imports
# ! pip install -U -q transformers accelerate datasets
import os
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
import re
import numpy as np
from matplotlib import pyplot as plt

from transformers import AutoProcessor, PaliGemmaForConditionalGeneration, PaliGemmaProcessor
from datasets import load_dataset

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory

df = pd.read_csv("/kaggle/input/riscm-dataset/filtered_captions_fixed.csv")
display(df.head())


,source,split,image,filtered_caption
0,NWPU,test,NWPU_31430.jpg,A plane is parked on the runway next to the gr...
1,NWPU,test,NWPU_31431.jpg,Four planes of different sizes were on the mar...
2,NWPU,test,NWPU_31432.jpg,A plane parked in a line on the airport with s...
3,NWPU,test,NWPU_31433.jpg,Two planes of different sizes are neatly parke...
4,NWPU,test,NWPU_31434.jpg,Two aircraft were parked at the departure gates .


# Downloading Paligemma Weights

In [2]:
import kagglehub

# Download latest version
# path = kagglehub.model_download("google/paligemma-2/transformers/paligemma2-3b-pt-224")
# if you have downloaded weights before, use their path:
path = "/kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1"

print("Path to model files:", path)

Path to model files: /kaggle/input/paligemma-2/transformers/paligemma2-3b-pt-224/1


# Modeling and LoRA Param Configs

In [ ]:
import torch
from transformers import PaliGemmaForConditionalGeneration, PaliGemmaProcessor
from peft import get_peft_model, LoraConfig, TaskType

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
base_model = PaliGemmaForConditionalGeneration.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    local_files_only=True
).to(device)
processor = PaliGemmaProcessor.from_pretrained(path, local_files_only=True)

# The training and evaluation is done for Lora-1 and Lora-2 Hyperparams

# Lora-1 Hyperparams
lora_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.2,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj"]
)

# Lora-2 Hyperparams
# lora_config = LoraConfig(
#     r=2,
#     lora_alpha=4,
#     lora_dropout=0.1,
#     bias="none",
#     task_type=TaskType.CAUSAL_LM,
#     target_modules=["q_proj", "o_proj"]
# )

model = get_peft_model(base_model, lora_config)

# Total trainable parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

# Freezing everything but the attention layers

for name, param in model.named_parameters():
    if "attn" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False
        
# Trainable attention layer parameters
attn_params = sum(
    p.numel() for name, p in model.named_parameters()
    if p.requires_grad and "attn" in name
)

print(f"Total trainable parameters: {total_params}")
print(f"Trainable attention parameters: {attn_params}")


2025-05-14 09:42:25.093135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747215745.116327     137 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747215745.123339     137 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
batch_size = 1
learning_rate = 1e-5

# Dataset Preaparation and Data Loading

In [ ]:
import pandas as pd
from datasets import Dataset

# Split data according to train-val-test and remove unused columns (such split and source after preparing datasets)
train_df = df[df["split"] == "train"].reset_index(drop=True)
val_df = df[df["split"] == "val"].reset_index(drop=True)
test_df = df[df["split"] == "test"].reset_index(drop=True)
train_df = train_df.drop(columns=["source", "split"])
val_df = val_df.drop(columns=["source", "split"])
test_df = test_df.drop(columns=["source", "split"])
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)


In [ ]:
from PIL import Image

image_dir = "/kaggle/input/riscm-dataset/RISCM/resized"

# Create Data Loader
def collate_train_fn(examples):
    images = [Image.open(os.path.join(image_dir, ex["image"])).convert("RGB") for ex in examples]
    prompt = ["caption the image" for _ in examples]
    suffix = [example["filtered_caption"] for example in examples]

    inputs = processor(
        images=images,
        text=prompt,
        suffix=suffix,
        return_tensors="pt",
        padding="longest",
    )

    inputs = inputs.to(torch.bfloat16).to(device)
    return inputs


def collate_test_fn(examples):
    images = [Image.open(os.path.join(image_dir, ex["image"])).convert("RGB") for ex in examples]

    prompt = ["caption the image" for _ in examples]

    inputs = processor(
        images=images,
        text=prompt,
        return_tensors="pt",
        padding="longest",
    )

    inputs = inputs.to(torch.bfloat16).to(device)
    return inputs


train_dataloader = DataLoader(
    train_dataset,
    collate_fn=collate_train_fn,
    batch_size=batch_size,
    shuffle=True,
)

val_dataloader = DataLoader(
    val_dataset,
    collate_fn=collate_train_fn,
    batch_size=batch_size,
    shuffle=False,
)

test_dataloader = DataLoader(
    test_dataset,
    collate_fn=collate_test_fn,
    batch_size=batch_size,
    shuffle=False,
)

# Wandb Configs

In [ ]:
import wandb
import os

# Set W&B API key in the environment
os.environ["WANDB_API_KEY"] = "1640c7993fb7d40efc5ce820586098b3f8a569f4"

# Login to wandb
wandb.login()

# Choose the run name according to the lora param configuration
wandb.init(reinit=True, project="DI725_Project", name="paligemma-lora-ft-kaggle-lora1")
# wandb.init(reinit=True, project="DI725_Project", name="paligemma-lora-ft-kaggle-lora2")

model = model.to(dtype=torch.bfloat16, device=device)

# Train and Validation Loop

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
from collections import deque
from transformers import get_linear_schedule_with_warmup

NUMBER_OF_TRAIN_LOSS_LOG = 100
NUM_EPOCHS = 3

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate,weight_decay = 1e-3)
num_training_steps = len(train_dataloader) * NUM_EPOCHS
num_warmup_steps = int(0.1 * num_training_steps)  # 10% warmup

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)

model.train()
loss_window = deque(maxlen=NUMBER_OF_TRAIN_LOSS_LOG)

for epoch in range(NUM_EPOCHS):
    total_train_loss = 0
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch+1}")
    # Training Loop
    for idx, batch in progress_bar:
        # batch = {k: v.to(torch.bfloat16).to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_train_loss += loss.item()


        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        loss_window.append(loss.item())

        # print(f"Epoch: {epoch+1} Iter: {idx} Train Loss: {loss.item():.4f}")

        if idx % NUMBER_OF_TRAIN_LOSS_LOG == 0 and len(loss_window) == NUMBER_OF_TRAIN_LOSS_LOG:
            avg_last_runs = sum(loss_window) / NUMBER_OF_TRAIN_LOSS_LOG
            wandb.log({
                "train_loss": avg_last_runs,
                "step": epoch * len(train_dataloader) + idx,
                "learning_rate": scheduler.get_last_lr()[0]
            })
    # Validate the model per end of epochs
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for val_batch in val_dataloader:
            val_outputs = model(**val_batch)
            val_loss = val_outputs.loss
            total_val_loss += val_loss.item()

    # Calculate average losses
    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_val_loss = total_val_loss / len(val_dataloader)
    scheduler.step(avg_val_loss)
    wandb.log({
        "val_loss": avg_val_loss,
        "epoch": epoch + 1,
    })
    for param_group in optimizer.param_groups:
      print(f"Epoch {epoch+1} | Avg Train Loss: {avg_train_loss:.4f} | Avg Val Loss: {avg_val_loss:.4f} Learning Rate: {param_group['lr']:.6f}\n")

    model.train()


# Save the model weights after training

In [ ]:
import os
import wandb

# Choose the save path according to the lora config (1 or 2)

# Define save path
final_output_dir = "/kaggle/working/model"
os.makedirs(final_output_dir, exist_ok=True)

# Save model and processor
model.save_pretrained(final_output_dir)
processor.save_pretrained(final_output_dir)

# Create and log W&B artifact
artifact = wandb.Artifact("paligemma-lora1-model", type="model")
artifact.add_dir(final_output_dir)
wandb.log_artifact(artifact)

# # Define save path
# final_output_dir = "/kaggle/working/model"
# os.makedirs(final_output_dir, exist_ok=True)

# # Save model and processor
# model.save_pretrained(final_output_dir)
# processor.save_pretrained(final_output_dir)

# # Create and log W&B artifact
# artifact = wandb.Artifact("paligemma-lora2-model", type="model")
# artifact.add_dir(final_output_dir)
# wandb.log_artifact(artifact)


# Test the model on the test set and log bleu, rouge and meteor metrics

In [ ]:
import os
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
import torch
import wandb
from evaluate import load

# Load evaluation metrics
bleu = load("bleu")
rouge = load("rouge")
meteor = load("meteor")

# Set model to eval mode
model.eval()
predictions = []
references = []

# === Helper Function ===
def read_image(path):
    image = Image.open(path).convert("RGB")
    image = np.array(image)
    if image.shape[-1] == 4:
        image = image[:, :, :3]
    return image
# Evaluate only on test_df
for i, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating"):
    image_path = os.path.join("./RISCM/resized", row["image"])
    try:
        image = read_image(image_path)
    except Exception as e:
        print(f"❌ Skipped {image_path}: {e}")
        continue

    input_text = "caption the image\n"
    inputs = processor(
        text=input_text,
        images=image,
        do_convert_rgb=True,
        padding="longest",
        return_tensors="pt"
    ).to(base_model.device)

    inputs = inputs.to(dtype=base_model.dtype)

    with torch.no_grad():
        output = base_model.generate(**inputs, max_new_tokens=64)

    decoded = processor.decode(output[0], skip_special_tokens=True)
    cleaned = decoded.lower().replace("caption the image", "").strip(" :.-").strip()

    predictions.append(cleaned)
    references.append(row["filtered_caption"])
    
# Compute final scores
bleu_refs = [[ref] for ref in references]
bleu_score = bleu.compute(predictions=predictions, references=bleu_refs)["bleu"]
rouge_score = rouge.compute(predictions=predictions, references=references)["rougeL"]
meteor_score = meteor.compute(predictions=predictions, references=references)["meteor"]

# Log to wandb
wandb.log({
    "test_bleu": bleu_score,
    "test_rougeL": rouge_score,
    "test_meteor": meteor_score
})

# Log sample predictions
wandb_table = wandb.Table(columns=["Prediction", "Ground Truth"])
for pred, ref in zip(predictions[:50], references[:50]):
    wandb_table.add_data(pred, ref)

wandb.log({"test_predictions_table": wandb_table})
